In [27]:
# importing libraries
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
import scipy
from scipy.stats import skew
from sklearn.svm import SVC
from sklearn import utils
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path='drive/MyDrive/CogAge Dataset/bbh/'

In [42]:
train_msAcc = np.load(path+"training/"+"trainMSAccelerometer.npy")
train_msGyro = np.load(path+"training/"+"trainMSGyroscope.npy")
train_jinsAcc = np.load(path+"training/"+"trainJinsAccelerometer.npy")
train_jinsGyro = np.load(path+"training/"+"trainJinsGyroscope.npy")
train_Acc = np.load(path+"training/"+"trainAccelerometer.npy")
train_Gyro = np.load(path+"training/"+"trainGyroscope.npy")
train_Gravity = np.load(path+"training/"+"trainGravity.npy")
train_Magneto = np.load(path+"training/"+"trainMagnetometer.npy")
train_LinearAcc = np.load(path+"training/"+"trainLinearAcceleration.npy")
train_labels = np.load(path+"training/"+"trainLabels.npy")

test_msAcc = np.load(path+"testing/"+"testMSAccelerometer.npy")
test_msGyro = np.load(path+"testing/"+"testMSGyroscope.npy")
test_jinsAcc = np.load(path+"testing/"+"testJinsAccelerometer.npy")
test_jinsGyro = np.load(path+"testing/"+"testJinsGyroscope.npy")
test_Acc = np.load(path+"testing/"+"testAccelerometer.npy")
test_Gyro = np.load(path+"testing/"+"testGyroscope.npy")
test_Gravity = np.load(path+"testing/"+"testGravity.npy")
test_Magneto = np.load(path+"testing/"+"testMagnetometer.npy")
test_LinearAcc = np.load(path+"testing/"+"testLinearAcceleration.npy")
test_labels = np.load(path+"testing/"+"testLabels.npy")

In [43]:
#cheching shape of the dataset
print(train_msAcc.shape)
print(train_msGyro.shape)
print(train_jinsAcc.shape)
print(train_jinsGyro.shape)
print(train_Acc.shape)
print(train_Gyro.shape)
print(train_LinearAcc.shape)
print(train_Magneto.shape)
print(train_Gravity.shape)
print(train_labels.shape)

print(test_msAcc.shape)
print(test_msGyro.shape)
print(test_jinsAcc.shape)
print(test_jinsGyro.shape)
print(test_Acc.shape)
print(test_Gyro.shape)
print(test_LinearAcc.shape)
print(test_Magneto.shape)
print(test_Gravity.shape)
print(test_labels.shape)

(2284, 268, 3)
(2284, 268, 3)
(2284, 80, 3)
(2284, 80, 3)
(2284, 800, 3)
(2284, 800, 3)
(2284, 800, 3)
(2284, 200, 3)
(2284, 800, 3)
(2284,)
(2288, 268, 3)
(2288, 268, 3)
(2288, 80, 3)
(2288, 80, 3)
(2288, 800, 3)
(2288, 800, 3)
(2288, 800, 3)
(2288, 200, 3)
(2288, 800, 3)
(2288,)


**Feature Extraction**


In [44]:
#train features
num_features = 7
train_num_examples = np.shape(train_labels)[0]
train_features = np.zeros((train_num_examples, 9*num_features, 3))
#ms_accelerometer
train_features[:,0,:] = np.mean(train_msAcc, axis=1)
train_features[:,1,:] = np.max(train_msAcc, axis=1)
train_features[:,2,:] = np.min(train_msAcc, axis=1)
train_features[:,3,:] = np.std(train_msAcc, axis=1)
train_features[:,4,:] = np.percentile(train_msAcc,70, axis=1)
train_features[:,5,:] = scipy.stats.skew(train_msAcc, axis=1)
train_features[:,6,:] = np.var(train_msAcc, axis=1)
#ms_gyroscope
train_features[:,7,:] = np.mean(train_msGyro, axis=1)
train_features[:,8,:] = np.max(train_msGyro, axis=1)
train_features[:,9,:] = np.min(train_msGyro, axis=1)
train_features[:,10,:] = np.std(train_msGyro, axis=1)
train_features[:,11,:] = np.percentile(train_msGyro, 70,axis=1)
train_features[:,12,:] = scipy.stats.skew(train_msGyro, axis=1)
train_features[:,13,:] = np.var(train_msGyro, axis=1)
#jins_Accelerometer
train_features[:,14,:] = np.mean(train_jinsAcc, axis=1)
train_features[:,15,:] = np.max(train_jinsAcc, axis=1)
train_features[:,16,:] = np.min(train_jinsAcc, axis=1)
train_features[:,17,:] = np.std(train_jinsAcc, axis=1)
train_features[:,18,:] = np.percentile(train_jinsAcc,70, axis=1)
train_features[:,19,:] = scipy.stats.skew(train_jinsAcc, axis=1)
train_features[:,20,:] = np.var(train_jinsAcc, axis=1)
#jins_Gyroscope
train_features[:,21,:] = np.mean(train_jinsGyro, axis=1)
train_features[:,22,:] = np.max(train_jinsGyro, axis=1)
train_features[:,23,:] = np.min(train_jinsGyro, axis=1)
train_features[:,24,:] = np.std(train_jinsGyro, axis=1)
train_features[:,25,:] = np.percentile(train_jinsGyro,70, axis=1)
train_features[:,26,:] = scipy.stats.skew(train_jinsGyro, axis=1)
train_features[:,27,:] = np.var(train_jinsGyro, axis=1)
#Accelerometer
train_features[:,28,:] = np.mean(train_Acc, axis=1)
train_features[:,29,:] = np.max(train_Acc, axis=1)
train_features[:,30,:] = np.min(train_Acc, axis=1)
train_features[:,31,:] = np.std(train_Acc, axis=1)
train_features[:,32,:] = np.percentile(train_Acc,70, axis=1)
train_features[:,33,:] = scipy.stats.skew(train_Acc, axis=1)
train_features[:,34,:] = np.var(train_Acc, axis=1)
#Gyroscope
train_features[:,35,:] = np.mean(train_Gyro, axis=1)
train_features[:,36,:] = np.max(train_Gyro, axis=1)
train_features[:,37,:] = np.min(train_Gyro, axis=1)
train_features[:,38,:] = np.std(train_Gyro, axis=1)
train_features[:,39,:] = np.percentile(train_Gyro,70, axis=1)
train_features[:,40,:] = scipy.stats.skew(train_Gyro, axis=1)
train_features[:,41,:] = np.var(train_Gyro, axis=1)
#Magnetometer
train_features[:,42,:] = np.mean(train_Magneto, axis=1)
train_features[:,43,:] = np.max(train_Magneto, axis=1)
train_features[:,44,:] = np.min(train_Magneto, axis=1)
train_features[:,45,:] = np.std(train_Magneto, axis=1)
train_features[:,46,:] = np.percentile(train_Magneto,70, axis=1)
train_features[:,47,:] = scipy.stats.skew(train_Magneto, axis=1)
train_features[:,48,:] = np.var(train_Magneto, axis=1)
#Gravity
train_features[:,49,:] = np.mean(train_Gravity, axis=1)
train_features[:,50,:] = np.max(train_Gravity, axis=1)
train_features[:,51,:] = np.min(train_Gravity, axis=1)
train_features[:,52,:] = np.std(train_Gravity, axis=1)
train_features[:,53,:] = np.percentile(train_Gravity,70, axis=1)
train_features[:,54,:] = scipy.stats.skew(train_Gravity, axis=1)
train_features[:,55,:] = np.var(train_Gravity, axis=1)
#Linear Accelerometer
train_features[:,56,:] = np.mean(train_LinearAcc, axis=1)
train_features[:,57,:] = np.max(train_LinearAcc, axis=1)
train_features[:,58,:] = np.min(train_LinearAcc, axis=1)
train_features[:,59,:] = np.std(train_LinearAcc, axis=1)
train_features[:,60,:] = np.percentile(train_LinearAcc,70, axis=1)
train_features[:,61,:] = scipy.stats.skew(train_LinearAcc, axis=1)
train_features[:,62,:] = np.var(train_LinearAcc, axis=1)


In [45]:
#test features
test_num_examples = np.shape(test_labels)[0]
test_features = np.zeros((test_num_examples, 9*num_features, 3))


test_features[:,0,:] = np.mean(test_msAcc, axis=1)
test_features[:,1,:] = np.max(test_msAcc, axis=1)
test_features[:,2,:] = np.min(test_msAcc, axis=1)
test_features[:,3,:] = np.std(test_msAcc, axis=1)
test_features[:,4,:] = np.percentile(test_msAcc,70, axis=1)
test_features[:,5,:] = scipy.stats.skew(test_msAcc, axis=1)
test_features[:,6,:] = np.var(test_msAcc, axis=1)


test_features[:,7,:] = np.mean(test_msGyro, axis=1)
test_features[:,8,:] = np.max(test_msGyro, axis=1)
test_features[:,9,:] = np.min(test_msGyro, axis=1)
test_features[:,10,:] = np.std(test_msGyro, axis=1)
test_features[:,11,:] = np.percentile(test_msGyro, 70,axis=1)
test_features[:,12,:] = scipy.stats.skew(test_msGyro, axis=1)
test_features[:,13,:] = np.var(test_msGyro, axis=1)

test_features[:,14,:] = np.mean(test_jinsAcc, axis=1)
test_features[:,15,:] = np.max(test_jinsAcc, axis=1)
test_features[:,16,:] = np.min(test_jinsAcc, axis=1)
test_features[:,17,:] = np.std(test_jinsAcc, axis=1)
test_features[:,18,:] = np.percentile(test_jinsAcc,70, axis=1)
test_features[:,19,:] = scipy.stats.skew(test_jinsAcc, axis=1)
test_features[:,20,:] = np.var(test_jinsAcc, axis=1)

test_features[:,21,:] = np.mean(test_jinsGyro, axis=1)
test_features[:,22,:] = np.max(test_jinsGyro, axis=1)
test_features[:,23,:] = np.min(test_jinsGyro, axis=1)
test_features[:,24,:] = np.std(test_jinsGyro, axis=1)
test_features[:,25,:] = np.percentile(test_jinsGyro,70, axis=1)
test_features[:,26,:] = scipy.stats.skew(test_jinsGyro, axis=1)
test_features[:,27,:] = np.var(test_jinsGyro, axis=1)

test_features[:,28,:] = np.mean(test_Acc, axis=1)
test_features[:,29,:] = np.max(test_Acc, axis=1)
test_features[:,30,:] = np.min(test_Acc, axis=1)
test_features[:,31,:] = np.std(test_Acc, axis=1)
test_features[:,32,:] = np.percentile(test_Acc,70, axis=1)
test_features[:,33,:] = scipy.stats.skew(test_Acc, axis=1)
test_features[:,34,:] = np.var(test_Acc, axis=1)


test_features[:,35,:] = np.mean(test_Gyro, axis=1)
test_features[:,36,:] = np.max(test_Gyro, axis=1)
test_features[:,37,:] = np.min(test_Gyro, axis=1)
test_features[:,38,:] = np.std(test_Gyro, axis=1)
test_features[:,39,:] = np.percentile(test_Gyro,70, axis=1)
test_features[:,40,:] = scipy.stats.skew(test_Gyro, axis=1)
test_features[:,41,:] = np.var(test_Gyro, axis=1)

test_features[:,42,:] = np.mean(test_Magneto, axis=1)
test_features[:,43,:] = np.max(test_Magneto, axis=1)
test_features[:,44,:] = np.min(test_Magneto, axis=1)
test_features[:,45,:] = np.std(test_Magneto, axis=1)
test_features[:,46,:] = np.percentile(test_Magneto,70, axis=1)
test_features[:,47,:] = scipy.stats.skew(test_Magneto, axis=1)
test_features[:,48,:] = np.var(test_Magneto, axis=1)

test_features[:,49,:] = np.mean(test_Gravity, axis=1)
test_features[:,50,:] = np.max(test_Gravity, axis=1)
test_features[:,51,:] = np.min(test_Gravity, axis=1)
test_features[:,52,:] = np.std(test_Gravity, axis=1)
test_features[:,53,:] = np.percentile(test_Gravity,70, axis=1)
test_features[:,54,:] = scipy.stats.skew(test_Gravity, axis=1)
test_features[:,55,:] = np.var(test_Gravity, axis=1)

test_features[:,56,:] = np.mean(test_LinearAcc, axis=1)
test_features[:,57,:] = np.max(test_LinearAcc, axis=1)
test_features[:,58,:] = np.min(test_LinearAcc, axis=1)
test_features[:,59,:] = np.std(test_LinearAcc, axis=1)
test_features[:,60,:] = np.percentile(test_LinearAcc,70, axis=1)
test_features[:,61,:] = scipy.stats.skew(test_LinearAcc, axis=1)
test_features[:,62,:] = np.var(test_LinearAcc, axis=1)


In [46]:
print(np.shape(train_features))
train_features_reshaped = np.reshape(train_features, (np.shape(train_features)[0], np.shape(train_features)[1]*np.shape(train_features)[2] ))
print(np.shape(train_features_reshaped))


print(np.shape(test_features))
test_features_reshaped = np.reshape(test_features, (np.shape(test_features)[0], np.shape(test_features)[1]*np.shape(test_features)[2] ))
print(np.shape(test_features_reshaped))


(2284, 63, 3)
(2284, 189)
(2288, 63, 3)
(2288, 189)


In [47]:
#classification using SVM
classfier = SVC( c=1.0,kernel='linear')  
classfier.fit(train_features_reshaped,train_labels)
estimatedLabels = classfier.predict(test_features_reshaped)
#Evaluation of model
accuracy = accuracy_score(test_labels,estimatedLabels)*100
weightedF1 = f1_score(test_labels,estimatedLabels,average='weighted')
averageF1 = f1_score(test_labels,estimatedLabels,average='macro')
allF1Scores = f1_score(test_labels,estimatedLabels,average=None)
conf_matrix = confusion_matrix(test_labels,estimatedLabels)
#  results
print('   Average F1-score = %.4f' % (averageF1))
print('   Test accuracy = %.2f %%' % (accuracy))
print('   Weighted F1-score = %.4f' % (weightedF1))
print('   All F1-scores:')
print(allF1Scores)
print('   Confusion Matrix:')
print(conf_matrix)

print('-------------------------------------------------------')



   C = 1.0 
   Average F1-score = 0.4686
   Test accuracy = 47.07 %
   Weighted F1-score = 0.4674
   All F1-scores:
[0.75609756 0.7037037  0.22222222 0.57142857 0.38       0.30434783
 0.40707965 0.22222222 0.04819277 0.43478261 0.5        0.51428571
 0.77142857 0.55913978 0.50746269 0.8        0.52941176 0.74725275
 0.45238095 0.37681159 0.38       0.24242424 0.39583333 0.1971831
 0.18604651 0.52380952 0.35897436 0.58139535 0.55       0.38961039
 0.38095238 0.46153846 0.55555556 0.56074766 0.20869565 0.51546392
 0.60869565 0.31746032 0.61333333 0.69811321 0.52525253 0.41269841
 0.4        0.55263158 0.69811321 0.39506173 0.29545455 0.27777778
 0.59090909 0.46153846 0.6        0.57142857 0.28235294 0.64197531
 0.53125   ]
   Confusion Matrix:
[[31  0  1 ...  0  0  0]
 [ 0 38  2 ...  1  0  0]
 [ 1  8 10 ...  2  0  0]
 ...
 [ 0  1  2 ... 12  0  0]
 [ 0  0  0 ...  0 26  0]
 [ 0  0  0 ...  0  0 17]]
-------------------------------------------------------


Neural Network attempt


In [50]:
train_msAcc = np.reshape(train_msAcc, (train_msAcc.shape[0],train_msAcc.shape[1]*train_msAcc.shape[2]))
train_msGyro = np.reshape(train_msGyro, (train_msGyro.shape[0],train_msGyro.shape[1]*train_msGyro.shape[2]))
train_jinsAcc = np.reshape(train_jinsAcc, (train_jinsAcc.shape[0],train_jinsAcc.shape[1]*train_jinsAcc.shape[2]))
train_jinsGyro = np.reshape(train_jinsGyro, (train_jinsGyro.shape[0],train_jinsGyro.shape[1]*train_jinsGyro.shape[2]))
train_Acc = np.reshape(train_Acc, (train_Acc.shape[0],train_Acc.shape[1]*train_Acc.shape[2]))
train_Gyro = np.reshape(train_Gyro, (train_Gyro.shape[0],train_Gyro.shape[1]*train_Gyro.shape[2]))
train_Magneto = np.reshape(train_Magneto, (train_Magneto.shape[0],train_Magneto.shape[1]*train_Magneto.shape[2]))
train_LinearAcc = np.reshape(train_LinearAcc, (train_LinearAcc.shape[0],train_LinearAcc.shape[1]*train_LinearAcc.shape[2]))
train_Gravity = np.reshape(train_Gravity, (train_Gravity.shape[0],train_Gravity.shape[1]*train_Gravity.shape[2]))

test_msAcc = np.reshape(test_msAcc, (test_msAcc.shape[0],test_msAcc.shape[1]*test_msAcc.shape[2]))
test_msGyro = np.reshape(test_msGyro, (test_msGyro.shape[0],test_msGyro.shape[1]*test_msGyro.shape[2]))
test_jinsAcc = np.reshape(test_jinsAcc, (test_jinsAcc.shape[0],test_jinsAcc.shape[1]*test_jinsAcc.shape[2]))
test_jinsGyro = np.reshape(test_jinsGyro, (test_jinsGyro.shape[0],test_jinsGyro.shape[1]*test_jinsGyro.shape[2]))
test_Acc = np.reshape(test_Acc, (test_Acc.shape[0],test_Acc.shape[1]*test_Acc.shape[2]))
test_Gyro = np.reshape(test_Gyro, (test_Gyro.shape[0],test_Gyro.shape[1]*test_Gyro.shape[2]))
test_Magneto = np.reshape(test_Magneto, (test_Magneto.shape[0],test_Magneto.shape[1]*test_Magneto.shape[2]))
test_LinearAcc = np.reshape(test_LinearAcc, (test_LinearAcc.shape[0],test_LinearAcc.shape[1]*test_LinearAcc.shape[2]))
test_Gravity = np.reshape(test_Gravity, (test_Gravity.shape[0],test_Gravity.shape[1]*test_Gravity.shape[2]))

print(train_msAcc.shape)
print(train_msGyro.shape)
print(train_jinsAcc.shape)
print(train_jinsGyro.shape)
print(train_Acc.shape)
print(train_Gyro.shape)
print(train_Magneto.shape)
print(train_LinearAcc.shape)
print(train_Gravity.shape)
print(test_msAcc.shape)
print(test_msGyro.shape)

(2284, 804)
(2284, 804)
(2284, 240)
(2284, 240)
(2284, 2400)
(2284, 2400)
(2284, 600)
(2284, 2400)
(2284, 2400)
(2288, 804)
(2288, 804)


In [51]:
print(train_labels)
lbl36 = train_labels==36
train_labels2 = lbl36.astype(int)
print(train_labels2)

print(test_labels)
lbl36 = test_labels==36
test_labels2 = lbl36.astype(int)
print(test_labels2)


[47 47 22 ... 33 29 28]
[0 0 0 ... 0 0 0]
[17 10 34 ... 29 34 10]
[0 0 0 ... 0 0 0]


In [13]:
# class CogAgeDataset(Dataset):
#     def __init__(self, cd_train_features,cd_labels ):
#         self.test_features_reshaped = cd_train_features
#         self.labels = cd_labels

#     def __len__(self):
#         return len(self.train_features)

#     def __getitem__(self, index):
#         return (self.train_features[index],self.labels[index])

In [14]:
# class ActivityMonitoring_FC(nn.Module):

#     def __init__(self):
#         super(ActivityMonitoring_FC, self).__init__()
        
#         self.fc11 = nn.Linear(189 ,2048)  #acc # flatten inpu dimension
        
#         self.fc21 = nn.Linear(2048 , 1024)
        
#         self.fc3 = nn.Linear(1024 , 512)
#         self.fc4 = nn.Linear(512 , 64)
#         self.output = nn.Linear(64 , 9)

#         self.dropout1 = nn.Dropout(p=0.3)
#         self.dropout2 = nn.Dropout(p=0.3)

#     def forward(self, train_features):

#         #print("input shape:", train_features.shape)
#         train_features = self.fc11(train_features) #z-vector
#         train_features = F.relu(train_features)
#         train_features = self.fc21(train_features)
#         train_features = F.relu(train_features)

#         #print("train_features.shape:", train_features.shape)
#         act = F.relu(self.fc3(train_features))
#         #print("act shape after fc3: ", act.shape)
        
#         act = self.dropout1(act)
#         act = F.relu(self.fc4(act))
#         #print("act shape after fc4: ", act.shape)
#         act = self.dropout2(act)
#         act_out = self.output(act)
#         #print("act_out shape after output layer: ", act_out.shape)

#         return act_out



In [52]:
class CogAgeDataset(Dataset):
    def __init__(self, cd_msAcc, cd_msGyro,cd_jinsAcc, cd_jinsGyro,cd_Acc, cd_Gyro,cd_Magneto,cd_LinearAcc, cd_Gravity, cd_labels ):
        self.msAcc = cd_msAcc
        self.msGyro = cd_msGyro
        self.labels = cd_labels
        self.jinsAcc = cd_jinsAcc
        self.jinsGyro = cd_jinsGyro
        self.Magneto = cd_Magneto
        self.Acc = cd_Acc
        self.Gyro = cd_Gyro
        self.LinearAcc = cd_LinearAcc
        self.Gravity = cd_Gravity
        self.Magneto = cd_Magneto

        
    def __len__(self):
        return len(self.msAcc)

    def __getitem__(self, index):
        return (self.Acc[index], self.Gyro[index], self.Magneto[index],self.LinearAcc[index], self.Gravity[index], self.jinsAcc[index],self.jinsGyro[index],self.msAcc[index], self.msGyro[index], self.labels[index])

In [53]:
class ActivityMonitoring_FC(nn.Module):

    def __init__(self):
        super(ActivityMonitoring_FC, self).__init__()
        
        self.fc11 = nn.Linear(2400 , 1024)  #ms_acc # flatten inpu dimension
        self.fc12 = nn.Linear(2400 , 1024) #ms_gyro 
        self.fc13 = nn.Linear(2400 , 1024) #jinsAcc
        self.fc14 = nn.Linear(600 , 1024) #jinsGYRO
        self.fc15 = nn.Linear(2400 , 1024) #Acc
        self.fc16 = nn.Linear(240 , 1024) #gyro
        self.fc17 = nn.Linear(240 , 1024) #mag
        self.fc18 = nn.Linear(804 , 1024) #LA
        self.fc19 = nn.Linear(804 , 1024) #grav


        
        self.fc21 = nn.Linear(1024 , 1024)
        self.fc22 = nn.Linear(1024 , 1024)
        self.fc23 = nn.Linear(1024 , 1024)
        self.fc24 = nn.Linear(1024 , 1024)
        self.fc25 = nn.Linear(1024 , 1024)
        self.fc26 = nn.Linear(1024 , 1024)
        self.fc27 = nn.Linear(1024 , 1024)
        self.fc28 = nn.Linear(1024 , 1024)
        self.fc29 = nn.Linear(1024 , 1024)
        
        self.fc3 = nn.Linear(1024 , 1024)
        self.fc4 = nn.Linear(1024 , 64)
        self.output = nn.Linear(64 , 2)

        self.dropout1 = nn.Dropout(p=0.9)
        self.dropout2 = nn.Dropout(p=0.9)

    def forward(self, msAcc, msGyro,jinsAcc,jinsGyro,Acc,Gyro,Magneto,LinearAcc,Gravity):

        print("input shape:", msAcc.shape)
        msAcc = self.fc11(msAcc) #z-vector
        msAcc = F.relu(msAcc)
        msAcc = self.fc21(msAcc)
        msAcc = F.relu(msAcc)

        print("msAcc.shape:", msAcc.shape)


        print("input shape:", msGyro.shape)
        msGyro = self.fc12(msGyro)
        msGyro = F.relu(msGyro)
        msGyro = self.fc22(msGyro)
        msGyro = F.relu(msGyro)

        print("msGyro.shape:", msGyro.shape)

        print("input shape:", jinsAcc.shape)
        jinsAcc = self.fc13(jinsAcc) #z-vector
        jinsAcc = F.relu(jinsAcc)
        jinsAcc = self.fc23(jinsAcc)
        jinsAcc = F.relu(jinsAcc)

        print("jinsAcc.shape:", jinsAcc.shape)


        print("input shape:", jinsGyro.shape)
        jinsGyro = self.fc14(jinsGyro)
        jinsGyro = F.relu(jinsGyro)
        jinsGyro = self.fc24(jinsGyro)
        jinsGyro = F.relu(jinsGyro)

        print("jinGyro.shape:", jinsGyro.shape)
        
        print("input shape:", Acc.shape)
        Acc = self.fc15(Acc) #z-vector
        Acc = F.relu(Acc)
        Acc = self.fc25(Acc)
        Acc = F.relu(Acc)

        print("Acc.shape:", Acc.shape)


        print("input shape:", Gyro.shape)
        Gyro = self.fc16(Gyro)
        Gyro = F.relu(Gyro)
        Gyro = self.fc26(Gyro)
        Gyro = F.relu(Gyro)

        print("Gyro.shape:", Gyro.shape)

        print("input shape:", Magneto.shape)
        Magneto = self.fc17(Magneto)
        Magneto = F.relu(Magneto)
        Magneto = self.fc27(Magneto)
        Magneto = F.relu(Magneto)

        print("Magneto.shape:", Magneto.shape)

        print("input shape:", LinearAcc.shape)
        LinearAcc = self.fc18(LinearAcc) #z-vector
        LinearAcc = F.relu(LinearAcc)
        LinearAcc = self.fc28(LinearAcc)
        LinearAcc = F.relu(LinearAcc)

        print("LinearAcc.shape:", LinearAcc.shape)


        print("input shape:", Gravity.shape)
        Gravity = self.fc19(Gravity)
        Gravity = F.relu(Gravity)
        Gravity = self.fc29(Gravity)
        Gravity = F.relu(Gravity)

        print("Gravity.shape:", Gravity.shape)
        
        

        joint_channels = torch.cat(( msAcc, msGyro,jinsAcc,jinsGyro,Acc,Gyro,Magneto,LinearAcc,Gravity), 0)
        print("joint_channels.shape after concat:", joint_channels.shape)
        # joint_channels = self.dropout1(joint_channels)
        # print("joint_channels.shape after concat:", joint_channels.shape)
        
        act = F.relu(self.fc3(joint_channels))
        print("act shape after fc3: ", act.shape)
        
        act = self.dropout1(act)
        act = F.relu(self.fc4(act))
        print("act shape after fc4: ", act.shape)
        act = self.dropout2(act)
        act_out = self.output(act)
        print("act_out shape after output layer: ", act_out.shape)

        return act_out



In [54]:
#defining loss_fn, optimizer, model
lr = 0.01
act_model = ActivityMonitoring_FC()
optimizer_act = optim.Adam(act_model.parameters(), lr=lr) #for state activities
loss_fn = nn.CrossEntropyLoss()
batch_size = 1024

In [55]:
train_dataset = CogAgeDataset(train_msAcc, train_msGyro,train_jinsAcc,train_jinsGyro,train_Acc,train_Gyro,train_LinearAcc,train_Magneto,train_Gravity, train_labels2)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = CogAgeDataset(test_msAcc, test_msGyro,test_jinsAcc,test_jinsGyro,test_Acc,test_Gyro,test_LinearAcc,test_Magneto,test_Gravity, test_labels2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [57]:
#training = 0
samples_tr = 0
correct_tr = 0
act_model.train()
n_epochs = 5
for epoch in range(n_epochs): #training loop
    batch_losses = []

    act_sample_tr = 0
    act_sample_ts = 0
    for msAcc_tr, msGyro_tr,jinsAcc_tr,jinsGyro_tr,Acc_tr,Gyro_tr,Magneto_try,LinearAcc_tr,Gravity_tr, labels_tr in train_loader:
            # print("Dimensions of each sensor modality:")
            # print(f"msAcc_tr: {msAcc_tr.shape}")
            # print(f"msGyro_tr: {msGyro_tr.shape}")
            # print(f"bbh_labels_tr: {labels_tr.shape}")

            
            #labels_tr  = labels_tr.to(device)
            
            optimizer_act.zero_grad()

            labels_predicted = act_model(msAcc_tr, msGyro_tr,jinsAcc_tr,jinsGyro_tr,Acc_tr,Gyro_tr,Magneto_try,LinearAcc_tr,Gravity_tr)
            #labels_predicted = labels_predicted.flatten()
            labels_tr = labels_tr.type(torch.LongTensor)

            _, predicted_tr = torch.max(labels_predicted,1)
            #print("predicted_tr.shape", predicted_tr.shape)   

            samples_tr += labels_tr.size(0)
            correct_tr += (predicted_tr[1] == labels_tr).sum().item()
            print('labes predicted',labels_predicted.shape)
            print('labes predicted-tr',labels_tr.shape)

            loss = loss_fn(labels_predicted, labels_tr)
            loss = loss.unsqueeze(0)
            epoch_loss = loss.item()
            print("Epoch, Loss:" , epoch,", ", epoch_loss)
            batch_losses.append(loss.item())
            
            loss.backward()
            optimizer_act.step()

    act_loss = sum(batch_losses)/len(batch_losses)
        #average_loss[1].append(bbh_loss)
    accr = 100.0 * correct_tr / samples_tr

    print(f"epoch: {epoch}")
    print(f"Training: batch_loss: {act_loss:.3f}, Accuracy: {accr:.3f}")
    #print(f"Training: batch_loss: {act_loss:.3f}, {state_correct_tr}/{state_samples_tr}, state_acc: {state_acc:.3f}%,  bbh_loss: {bbh_loss:.3f}, {bbh_correct_tr}/{bbh_samples_tr} , bbh_acc: {bbh_acc:.3f}%")

 

input shape: torch.Size([1024, 2400])
msAcc.shape: torch.Size([1024, 1024])
input shape: torch.Size([1024, 2400])
msGyro.shape: torch.Size([1024, 1024])
input shape: torch.Size([1024, 2400])
jinsAcc.shape: torch.Size([1024, 1024])
input shape: torch.Size([1024, 600])
jinGyro.shape: torch.Size([1024, 1024])
input shape: torch.Size([1024, 2400])
Acc.shape: torch.Size([1024, 1024])
input shape: torch.Size([1024, 240])
Gyro.shape: torch.Size([1024, 1024])
input shape: torch.Size([1024, 240])
Magneto.shape: torch.Size([1024, 1024])
input shape: torch.Size([1024, 804])
LinearAcc.shape: torch.Size([1024, 1024])
input shape: torch.Size([1024, 804])
Gravity.shape: torch.Size([1024, 1024])
joint_channels.shape after concat: torch.Size([9216, 1024])
act shape after fc3:  torch.Size([9216, 1024])
act shape after fc4:  torch.Size([9216, 64])
act_out shape after output layer:  torch.Size([9216, 2])
labes predicted torch.Size([9216, 2])
labes predicted-tr torch.Size([1024])


ValueError: ignored